In [46]:
import pandas as pd
import json

from pandas.io.json import json_normalize


quarter_1_2017_hourly = pd.read_csv('LA_20171_Hourly_Weekdays.csv')
quarter_1_2017_monthly = pd.read_csv('LA_20171_Monthly.csv')
# quarter_2_2017 = pd.read_csv('desktop/20172.csv')
# quarter_3_2017 = pd.read_csv('desktop/20173.csv')
# quarter_4_2017 = pd.read_csv('desktop/20174.csv')
# quarter_1_2018 = pd.read_csv('desktop/20181.csv')

def getFrequency(a):
    '''
    input:
    a: a dataframe read by pandas.
    output:
    a series with the most frequent 10 locations within either individual pickup location or destination
    '''
    sid = a.groupby(by='sourceid')['sourceid'].count().sort_values(ascending=False)
    sid_sd = sid.groupby(by='standard_deviation_travel_time')['standard_deviation_travel_time'].count().sort_values(ascending = False)
    return sid_sd[:15]
print(sid)
#getFrequency(quarter_1_2017_monthly)



{1: 1438, 2: 1460, 3: 1607, 4: 1759, 5: 655, 6: 750, 7: 2012, 8: 1528, 9: 1565, 10: 1757, 11: 1775, 12: 3796, 13: 1987, 14: 1606, 15: 3461, 16: 1260, 17: 1502, 18: 3747, 19: 2153, 20: 622, 21: 3689, 22: 3302, 23: 2148, 24: 2436, 25: 2551, 26: 2534, 27: 1403, 28: 1407, 29: 2323, 30: 1050, 31: 1157, 32: 192, 33: 1020, 34: 1404, 35: 1293, 36: 1326, 37: 920, 38: 864, 39: 1941, 40: 1118, 41: 1257, 42: 826, 43: 734, 44: 744, 45: 896, 46: 1437, 47: 1554, 48: 916, 49: 796, 50: 854, 51: 1555, 52: 1650, 53: 1023, 54: 667, 55: 648, 56: 812, 58: 3284, 59: 2485, 60: 2650, 61: 1000, 62: 2344, 63: 1115, 64: 829, 65: 703, 66: 662, 67: 871, 68: 464, 69: 753, 70: 854, 71: 881, 72: 968, 73: 864, 74: 470, 75: 609, 76: 2412, 77: 3782, 78: 1278, 79: 1337, 80: 1526, 81: 3626, 82: 1585, 83: 1183, 84: 1243, 85: 3906, 86: 1442, 87: 1337, 88: 1150, 89: 1106, 90: 1342, 91: 879, 92: 810, 93: 758, 94: 877, 95: 1191, 96: 1094, 97: 938, 98: 2463, 99: 1735, 100: 2558, 101: 2422, 102: 2369, 103: 2256, 104: 1057, 105: 1

In [ ]:
1697    6783
2423    6514
325     6357
1220    6255
1230    6217
412     6010
1233    5937
1222    5922

In [54]:
map_LA = pd.read_json('LA_map.json', orient='records')
map_LA_header = pd.DataFrame(map_LA.features.values.tolist())
map_LA_coor = dict(pd.DataFrame(map_LA_header.geometry.tolist())['coordinates']) # the index off by 1

map_LA_coor[46 - 1][0][0][0][1]

map_LA_coor

{0: [[[-118.11683, 34.107225],
   [-118.116311, 34.10648],
   [-118.115782, 34.106729],
   [-118.115717, 34.106606],
   [-118.115696, 34.106563],
   [-118.115653, 34.106468],
   [-118.115613, 34.106402],
   [-118.115488, 34.10611],
   [-118.115409, 34.10611],
   [-118.11532, 34.106112],
   [-118.115171, 34.106113],
   [-118.115174, 34.106068],
   [-118.115174, 34.105996],
   [-118.11517, 34.105924],
   [-118.115157, 34.105829],
   [-118.115142, 34.105758],
   [-118.115125, 34.105696],
   [-118.115762, 34.105696],
   [-118.115543, 34.105381],
   [-118.113094, 34.101877],
   [-118.11306, 34.10185],
   [-118.112989, 34.101707],
   [-118.114592, 34.100991],
   [-118.114876, 34.100861],
   [-118.116693, 34.099987],
   [-118.117786, 34.099464],
   [-118.120994, 34.097937],
   [-118.12254, 34.097193],
   [-118.124152, 34.096419],
   [-118.125194, 34.095935],
   [-118.126868, 34.09516],
   [-118.127179, 34.095604],
   [-118.127898, 34.096637],
   [-118.12827, 34.097221],
   [-118.128556, 34.09

In [48]:
lat = []
long = []
times = []
sid = dict(quarter_1_2017_monthly.groupby('sourceid')['sourceid'].count())
sourceids = []

for i in range(len(map_LA_coor)):
    if len(map_LA_coor[i][0][0]) == 2:
        long.append(map_LA_coor[i][0][0][0])
        lat.append(map_LA_coor[i][0][0][1])
        
    else:
        long.append(map_LA_coor[i][0][0][0][0])
        lat.append(map_LA_coor[i][0][0][0][1])
    if (i+1) in sid.keys():
        sourceids.append(sid[i+1])
    else:
        sourceids.append(0)


In [49]:
destid_morning = dict(quarter_1_2017_hourly[(quarter_1_2017_hourly['hod'] >= 6) & (quarter_1_2017_hourly['hod'] <= 9)].groupby('dstid')['dstid'].count())



In [50]:
destid_evening = dict(quarter_1_2017_hourly[(quarter_1_2017_hourly['hod'] >= 17) & (quarter_1_2017_hourly['hod'] <= 20)].groupby('dstid')['dstid'].count())


In [51]:
destid_noon = dict(quarter_1_2017_hourly[(quarter_1_2017_hourly['hod'] >= 12) & (quarter_1_2017_hourly['hod'] <= 14)].groupby('dstid')['dstid'].count())



In [52]:
destid_9am = dict(quarter_1_2017_hourly[(quarter_1_2017_hourly['hod'] == 9)].groupby('dstid')['dstid'].count())

## Processing the Morning and Evening data


In [53]:
dstids_morning = []
dstids_evening = []
destids_noon = []
destids_9am = []
for i in range(len(map_LA_coor)):
    if (i+1) in destid_morning.keys():
        dstids_morning.append(destid_morning[i+1])
    else:
        dstids_morning.append(0)
    if (i+1) in destid_evening.keys():
        dstids_evening.append(destid_evening[i+1])
    else:
        dstids_evening.append(0)
    if (i+1) in destid_noon.keys():
        destids_noon.append(destid_noon[i+1])
    else:
        destids_noon.append(0)
    if (i+1) in destid_9am.keys():
        destids_9am.append(destid_9am[i+1])
    else:
        destids_9am.append(0)

In [62]:
quarter_1_2017_monthly


,sourceid,dstid,month,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,1159,2294,2,1459.54,682.03,1361.45,1.41
1,1239,1494,2,1438.76,622.83,1325.06,1.49
2,1269,1194,2,1426.51,450.62,1357.36,1.37
3,1279,1094,2,805.02,470.66,710.69,1.60
4,1409,1289,1,1718.72,457.66,1663.87,1.29
5,1418,1199,1,1478.61,626.06,1375.01,1.44
6,1249,1394,2,1164.99,416.42,1100.63,1.39
7,374,358,1,1033.26,384.93,978.97,1.37
8,612,371,1,2377.02,876.07,2232.73,1.42
9,1259,1294,2,391.13,218.21,347.47,1.62


In [103]:
import pandas as pd 
import folium
from folium.plugins import HeatMap

max_amount = float(max(sid.values()))

hmap = folium.Map(location=[34.17225, -118.544728], zoom_start=7, )

hm_wide = HeatMap( list(zip(lat, long, sourceids)),
                   min_opacity=0.05,
                   max_val=max_amount,
                   radius=8, blur=4, 
                   max_zoom=1, 
                 )

# folium.GeoJson(district23).add_to(hmap)
hmap.add_child(hm_wide)

In [ ]:
# datat = quarter_1_2017[(quarter_1_2017.sourceid == 734) & (quarter_1_2017.month == 2)]

# map_LA = pd.read_json('LA_map.json', orient='records')

# map_LA_header = pd.DataFrame(map_LA.features.values.tolist())
# map_LA_coor = dict(pd.DataFrame(map_LA_header.geometry.tolist())['coordinates'])


# locations = dict(getFrequency(quarter_1_2017))

# long = []
# lat = []
# for i in range(len(map_LA_coor)):
#     map_LA_coor[i] = map_LA_coor[i][0][0]
#     print(map_LA_coor[i])

# for index, data in datat.iterrows():
#     value = data['mean_travel_time']/100
    
#     while value > 0:
#         if isinstance(map_LA_coor[data['dstid']][0], list):
#             long.append(map_LA_coor[data['dstid']][0][0])
#         else:
#             long.append(map_LA_coor[data['dstid']][0])
#         print(map_LA_coor[data['dstid']][0])
#         if isinstance(map_LA_coor[data['dstid']][1], list):
#             lat.append(map_LA_coor[data['dstid']][0][1])
#         else:
#             lat.append(map_LA_coor[data['dstid']][1])
            
#         value = value - 1

# print(lat)
# locations

In [104]:
hourly

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,1,2,3,295.21,259.18,239.67,1.77
1,1,4,21,151.66,112.14,120.24,2.08
2,1,7,8,699.09,212.16,670.63,1.33
3,2,4,20,130.32,105.30,99.98,2.12
4,2,7,7,454.84,201.44,418.10,1.51
5,3,2,1,93.31,81.87,66.91,2.41
6,3,4,19,150.35,115.39,120.98,1.95
7,3,7,6,531.77,213.75,495.76,1.44
8,4,2,0,110.53,104.97,81.35,2.19
9,4,7,5,478.66,166.58,452.53,1.39


In [ ]:
# import gmplot

# gmap = gmplot.GoogleMapPlotter(34.107225, -118.11683, 10)  
  
# # heatmap plot heating Type 
# # points on the Google map 
# gmap.heatmap( lat, long ) 
# print(lat)
  
# gmap.draw( "desktop/map2017.html" ) 


## Hourly: weekdays - Morning

In [121]:
import pandas as pd 
import folium
from folium.plugins import HeatMap

max_amount = float(max(sid.values()))

hmap = folium.Map(location=[34.17225, -118.544728], zoom_start=7, )

hm_wide = HeatMap( list(zip(lat, long, dstids_morning)),
                   min_opacity=0.05,
                   max_val=max_amount,
                   radius=3, blur=4, 
                   max_zoom=1, 
                 )

# folium.GeoJson(district23).add_to(hmap)
hmap.add_child(hm_wide)

## Hourly: Weekdays - evening

In [122]:
import pandas as pd 
import folium
from folium.plugins import HeatMap

max_amount = float(max(sid.values()))

hmap = folium.Map(location=[34.17225, -118.544728], zoom_start=7, )

hm_wide = HeatMap( list(zip(lat, long, dstids_evening)),
                   min_opacity=0.05,
                   max_val=max_amount,
                   radius=3, blur=4, 
                   max_zoom=1, 
                 )

# folium.GeoJson(district23).add_to(hmap)
hmap.add_child(hm_wide)

## Hourly: Weekdays - Noon

In [147]:
import pandas as pd 
import folium
from folium.plugins import HeatMap

max_amount = float(max(sid.values()))

hmap = folium.Map(location=[34.17225, -118.544728], zoom_start=7, )

hm_wide = HeatMap( list(zip(lat, long, destids_noon)),
                   min_opacity=0.1,
                   max_val=max_amount,
                   radius=3, blur=4, 
                   max_zoom=1, 
                 )

# folium.GeoJson(district23).add_to(hmap)
hmap.add_child(hm_wide)

## Hourly: weekdays - 9 am

In [128]:
import pandas as pd 
import folium
from folium.plugins import HeatMap

max_amount = float(max(sid.values()))

hmap = folium.Map(location=[34.17225, -118.544728], zoom_start=7, )

hm_wide = HeatMap( list(zip(lat, long, destids_9am)),
                   min_opacity=0.4,
                   max_val=max_amount,
                   radius=3, blur=4, 
                   max_zoom=1, 
                 )

# folium.GeoJson(district23).add_to(hmap)
hmap.add_child(hm_wide)

## Monthly: 2017 First Quarter

In [129]:
destid = dict(quarter_1_2017_monthly.groupby('dstid')['dstid'].count())
destids = []
for i in range(len(map_LA_coor)):
    
    if (i+1) in destid.keys():
        destids.append(destid[i+1])
    else:
        destids.append(0)

import pandas as pd 
import folium
from folium.plugins import HeatMap

max_amount = float(max(sid.values()))

hmap = folium.Map(location=[34.17225, -118.544728], zoom_start=7, )

hm_wide = HeatMap( list(zip(lat, long, destids)),
                   min_opacity=0.4,
                   max_val=max_amount,
                   radius=3, blur=4, 
                   max_zoom=1, 
                 )

# folium.GeoJson(district23).add_to(hmap)
hmap.add_child(hm_wide)

## Traffic Collision

In [6]:
import pandas as pd
traffic = pd.read_csv('Traffic_Collision.csv')


In [ ]:
traffic

In [7]:
traffic['Date Occurred'] = pd.to_datetime(traffic['Date Occurred'])
traffic['year'], traffic['month'] = traffic['Date Occurred'].dt.year, traffic['Date Occurred'].dt.month

traffic

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Address,Cross Street,Location,year,month
0,100119684,01/01/11042010,2010-11-04,1345,1,Central,159,997,TRAFFIC COLLISION,NaN,20.0,F,H,101.0,STREET,7TH ST,LAWRENCE ST,"(34.0348, -118.2362)",2010,11
1,101221255,01/01/8102010,2010-08-10,1900,12,77th Street,1211,997,TRAFFIC COLLISION,NaN,31.0,F,B,101.0,STREET,54TH ST,3RD AV,"(33.9932, -118.3203)",2010,8
2,101414426,01/01/6112010,2010-06-09,1925,14,Pacific,1488,997,TRAFFIC COLLISION,NaN,40.0,F,B,101.0,STREET,MANCHESTER BL,BELLANCA AV,"(33.9599, -118.3805)",2010,6
3,102109168,01/01/4062010,2010-04-06,1445,21,Topanga,2197,997,TRAFFIC COLLISION,1501,51.0,M,O,101.0,STREET,ROSITA AV,CORBIN AV,"(34.1585, -118.5633)",2010,4
4,100322445,01/01/9222010,2010-05-23,1805,3,Southwest,397,997,TRAFFIC COLLISION,NaN,32.0,F,H,101.0,STREET,43RD PL,WALTON AV,"(34.0055, -118.2948)",2010,5
5,101616651,01/01/9022010,2010-09-02,1520,16,Foothill,1667,997,TRAFFIC COLLISION,0605,45.0,F,W,101.0,STREET,9600 SUNLAND BL,NaN,"(34.246, -118.3622)",2010,9
6,101226806,01/01/10282010,2010-10-27,1910,12,77th Street,1251,997,TRAFFIC COLLISION,NaN,61.0,F,B,101.0,STREET,CRENSHAW BL,74TH ST,"(33.9727, -118.3309)",2010,10
7,101224882,01/01/9302010,2010-09-30,540,12,77th Street,1265,997,TRAFFIC COLLISION,NaN,70.0,M,B,101.0,STREET,NORMANDIE AV,MANCHESTER AV,"(33.96, -118.3003)",2010,9
8,111206659,01/01/2082011,2010-11-19,1440,12,77th Street,1215,997,TRAFFIC COLLISION,NaN,NaN,M,O,101.0,STREET,54TH ST,NORMANDIE AV,"(33.9932, -118.3002)",2010,11
9,101307375,01/01/2212010,2010-02-21,300,13,Newton,1361,997,TRAFFIC COLLISION,0605,44.0,F,O,101.0,STREET,100 W 52ND ST,NaN,"(33.9956, -118.274)",2010,2


In [23]:
traffic['lat'] = traffic['Location'].apply( lambda x: float(x.split(',')[0][1:]) )
traffic['lon'] = traffic['Location'].apply( lambda x: float(x.split(',')[1][:-1]) )

In [9]:
exam = [[[-118.355696,34.03392],[-118.356258,34.032499],[-118.356559,34.031948],[-118.35911,34.02549],[-118.360435,34.025483],[-118.361169,34.025552],[-118.365211,34.025927],[-118.362628,34.032474],[-118.362546,34.032893],[-118.362231,34.033774],[-118.361888,34.033748],[-118.361309,34.03372],[-118.361019,34.033712],[-118.360438,34.033709],[-118.359882,34.033722],[-118.355696,34.03392]]]

print(max(exam))

[[-118.355696, 34.03392], [-118.356258, 34.032499], [-118.356559, 34.031948], [-118.35911, 34.02549], [-118.360435, 34.025483], [-118.361169, 34.025552], [-118.365211, 34.025927], [-118.362628, 34.032474], [-118.362546, 34.032893], [-118.362231, 34.033774], [-118.361888, 34.033748], [-118.361309, 34.03372], [-118.361019, 34.033712], [-118.360438, 34.033709], [-118.359882, 34.033722], [-118.355696, 34.03392]]


In [10]:


traffic

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Victim Descent,Premise Code,Premise Description,Address,Cross Street,Location,year,month,lat,lon
0,100119684,01/01/11042010,2010-11-04,1345,1,Central,159,997,TRAFFIC COLLISION,NaN,...,H,101.0,STREET,7TH ST,LAWRENCE ST,"(34.0348, -118.2362)",2010,11,34.0348,-118.2362
1,101221255,01/01/8102010,2010-08-10,1900,12,77th Street,1211,997,TRAFFIC COLLISION,NaN,...,B,101.0,STREET,54TH ST,3RD AV,"(33.9932, -118.3203)",2010,8,33.9932,-118.3203
2,101414426,01/01/6112010,2010-06-09,1925,14,Pacific,1488,997,TRAFFIC COLLISION,NaN,...,B,101.0,STREET,MANCHESTER BL,BELLANCA AV,"(33.9599, -118.3805)",2010,6,33.9599,-118.3805
3,102109168,01/01/4062010,2010-04-06,1445,21,Topanga,2197,997,TRAFFIC COLLISION,1501,...,O,101.0,STREET,ROSITA AV,CORBIN AV,"(34.1585, -118.5633)",2010,4,34.1585,-118.5633
4,100322445,01/01/9222010,2010-05-23,1805,3,Southwest,397,997,TRAFFIC COLLISION,NaN,...,H,101.0,STREET,43RD PL,WALTON AV,"(34.0055, -118.2948)",2010,5,34.0055,-118.2948
5,101616651,01/01/9022010,2010-09-02,1520,16,Foothill,1667,997,TRAFFIC COLLISION,0605,...,W,101.0,STREET,9600 SUNLAND BL,NaN,"(34.246, -118.3622)",2010,9,34.246,-118.3622
6,101226806,01/01/10282010,2010-10-27,1910,12,77th Street,1251,997,TRAFFIC COLLISION,NaN,...,B,101.0,STREET,CRENSHAW BL,74TH ST,"(33.9727, -118.3309)",2010,10,33.9727,-118.3309
7,101224882,01/01/9302010,2010-09-30,540,12,77th Street,1265,997,TRAFFIC COLLISION,NaN,...,B,101.0,STREET,NORMANDIE AV,MANCHESTER AV,"(33.96, -118.3003)",2010,9,33.96,-118.3003
8,111206659,01/01/2082011,2010-11-19,1440,12,77th Street,1215,997,TRAFFIC COLLISION,NaN,...,O,101.0,STREET,54TH ST,NORMANDIE AV,"(33.9932, -118.3002)",2010,11,33.9932,-118.3002
9,101307375,01/01/2212010,2010-02-21,300,13,Newton,1361,997,TRAFFIC COLLISION,0605,...,O,101.0,STREET,100 W 52ND ST,NaN,"(33.9956, -118.274)",2010,2,33.9956,-118.274


In [37]:
quarter_1_2017_monthly = pd.read_csv('LA_20171_Monthly.csv')

In [38]:
map_LA = pd.read_json('LA_map.json', orient='records')
map_LA_header = pd.DataFrame(map_LA.features.values.tolist())
map_LA_coor = dict(pd.DataFrame(map_LA_header.geometry.tolist())['coordinates']) # the index off by 1

lat = []
long = []
times = []
sid = dict(quarter_1_2017_monthly.groupby('sourceid')['sourceid'].count())
sourceids = []

for i in range(len(map_LA_coor)):
    if len(map_LA_coor[i][0][0]) == 2:
        long.append(map_LA_coor[i][0][0][0])
        lat.append(map_LA_coor[i][0][0][1])
        
    else:
        long.append(map_LA_coor[i][0][0][0][0])
        lat.append(map_LA_coor[i][0][0][0][1])
    if (i+1) in sid.keys():
        sourceids.append(sid[i+1])
    else:
        sourceids.append(0)



In [44]:
map_LA = pd.read_json('LA_map.json', orient='records')
map_LA_header = pd.DataFrame(map_LA.features.values.tolist())
map_LA_coor = dict(pd.DataFrame(map_LA_header.geometry.tolist())['coordinates']) # the index off by 1

lat = []
long = []
times = []
sid = dict(quarter_1_2017_monthly.groupby('sourceid')['sourceid'].count())
sourceids = []

for i in range(len(map_LA_coor)):
    if len(map_LA_coor[i][0][0]) == 2:
        long.append(map_LA_coor[i][0][0][0])
        lat.append(map_LA_coor[i][0][0][1])
        
    else:
        long.append(map_LA_coor[i][0][0][0][0])
        lat.append(map_LA_coor[i][0][0][0][1])
    if (i+1) in sid.keys():
        sourceids.append(sid[i+1])
    else:
        sourceids.append(0)




In [24]:
exa = traffic['lat'][0]
exa

34.0348

In [35]:
lati = []
longti = []
for i in range(traffic.shape[0]):
    if traffic['lat'][i] == 0 or traffic['lon'][i] == 0 :
        break
    if traffic['year'][i] != 2017:
        break
    lati.append(traffic['lat'][i])
    longti.append(traffic['lon'][i])


In [45]:
from bokeh.plotting import figure, show, output_file


#colormap = {'setosa': 'red', 'versicolor': 'green', 'virginica': 'blue'}
#colors = [colormap[x] for x in flowers['species']]

output_file("draft_1.html")

p = figure(plot_width=400, plot_height=400)

p = figure(title = "Traffic Collisions vs. Popular Areas(Pick Up Location)")
p.xaxis.axis_label = 'Latitude'
p.yaxis.axis_label = 'Longtitude'

p.circle(lati, longti, size=2, color="red", alpha=0.5)
p.circle(lat, long, size = 2, color = "navy", alpha = 0.5)

show(p)

In [143]:
example = '(34.0348, -118.2362)'
result = example.split(',')
float(result[1][:-1])

(33.68 - 34.52), (-118.72, -117.76)

-118.2362

In [ ]:
## NEED to organize the different locations

positions = dict(quarter_1_2017_monthly.groupby('dstid')['dstid'].count())
destids = []
for i in range(len(map_LA_coor)):
    
    if (i+1) in destid.keys():
        destids.append(destid[i+1])
    else:
        destids.append(0)

import pandas as pd 
import folium
from folium.plugins import HeatMap

max_amount = float(max(sid.values()))

hmap = folium.Map(location=[34.17225, -118.544728], zoom_start=7, )

hm_wide = HeatMap( list(zip(lat, long, destids)),
                   min_opacity=0.4,
                   max_val=max_amount,
                   radius=3, blur=4, 
                   max_zoom=1, 
                 )

hmap.add_child(hm_wide)